In [1]:
import pandas as pd
import pymysql
import numpy as np

In [2]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [3]:
df = pd.read_sql_query("select * from g_master_data_joins order by date", __connect__())

In [4]:
original_df = df.copy()

In [5]:
# df = original_df.copy()

In [6]:
df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,Transaction GMV,Interest GMV,Price/Sale,Users 24hr Debank,Transactions Count Debank,Trading Volume Debank,Borrowing Volume Debank,Users life-to-date duneanlytics,CoinUrl,Tags
0,2020-12-21,Aave,AAVE,81.311937,9.757233e+08,2.154891e+08,1.199975e+07,1.600000e+07,1.600000e+07,1.723929e+09,...,1.190984e+06,3.366851e+08,72.158217,NaN,NaN,NaN,NaN,32549.0,https://aave.com,"defi,dao,yield-farming,three-arrows-capital-po..."
1,2020-12-21,AllianceBlock,ALBT,0.174965,0.000000e+00,7.007850e+06,0.000000e+00,1.000000e+09,1.000000e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://allianceblock.io,
2,2020-12-21,Amp,AMP,0.006596,6.361912e+07,3.698943e+05,9.645497e+09,1.200105e+09,9.239914e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://amptoken.org,
3,2020-12-21,Ampleforth,AMPL,0.820038,1.885767e+08,1.046414e+07,2.299609e+08,NaN,4.102788e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://ampleforth.org,"defi,rebase,arrington-xrp-capital,alameda-rese..."
4,2020-12-21,Anchor Neural World,ANW,0.105945,1.071682e+07,1.948454e+05,1.011550e+08,1.000000e+09,1.000000e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://an-va.com,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109940,2021-04-07,yOUcash,YOUC,0.045723,8.048407e+07,3.016991e+04,1.760253e+09,1.100000e+10,1.100000e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youengine.io,
109941,2021-04-07,YUSRA,YUSRA,0.443496,1.707955e+07,4.832632e+04,3.851117e+07,4.000000e+07,4.000000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://yusra.global,
109942,2021-04-07,Zano,ZANO,1.424225,1.507708e+07,9.044065e+04,1.058616e+07,NaN,1.261566e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://zano.org,privacy
109943,2021-04-07,Zenfuse,ZEFU,0.195110,1.073663e+07,1.533755e+06,5.502850e+07,NaN,2.000000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://zenfuse.io,


In [7]:
original_df.columns

Index(['date', 'Name', 'Symbol', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Debank', 'TVL Defipulse', 'Transaction Revenue',
       'Interest Revenue', 'Transaction Revenue Supply Side',
       'Interest Revenue Supply Side', 'Transaction Revenue Protocol',
       'Interest Revenue Protocol', 'Transaction GMV', 'Interest GMV',
       'Price/Sale', 'Users 24hr Debank', 'Transactions Count Debank',
       'Trading Volume Debank', 'Borrowing Volume Debank',
       'Users life-to-date duneanlytics', 'CoinUrl', 'Tags'],
      dtype='object')

In [8]:
df['TVL_max'] = df[['TVL Debank', 'TVL Defipulse']].max(axis=1)

In [9]:
df[df['Symbol'] == 'BTC']['Interest Revenue']

375      NaN
705      NaN
1538     NaN
2403     NaN
2743     NaN
          ..
105395   NaN
106129   NaN
106628   NaN
108054   NaN
109201   NaN
Name: Interest Revenue, Length: 108, dtype: float64

In [10]:
tran_gvm_df = df['Transaction GMV'].copy()
tran_gvm_df = tran_gvm_df.astype(object).where(pd.notnull(tran_gvm_df), 0)

int_gvm_df = df['Interest GMV'].copy()
int_gvm_df = int_gvm_df.astype(object).where(pd.notnull(int_gvm_df), 0)

df['GMV'] = (tran_gvm_df + int_gvm_df).replace(0, np.nan)

In [11]:
trading_volume_df = df['Trading Volume Debank'].copy()
trading_volume_df = trading_volume_df.astype(object).where(pd.notnull(trading_volume_df), 0)

borrowing_volume_df = df['Borrowing Volume Debank'].copy()
borrowing_volume_df = borrowing_volume_df.astype(object).where(pd.notnull(borrowing_volume_df), 0)

# trading volume + borrowing volume is a more broad category than 'dex' so 'defi' is better 
#df['Dex Volume DeBank'] = (trading_volume_df + borrowing_volume_df).replace(0, np.nan)
df['Defi Volume DeBank'] = (trading_volume_df + borrowing_volume_df).replace(0, np.nan)

# add defi volume max as max of gmv from tokenterminal and defi volume debank
df['Defi Volume_max'] = df[['GMV', 'Defi Volume DeBank']].max(axis=1)

In [12]:
df['Defi Volume DeBank']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
109940   NaN
109941   NaN
109942   NaN
109943   NaN
109944   NaN
Name: Defi Volume DeBank, Length: 109945, dtype: float64

In [13]:
# df['GMV'] = df['Transaction GMV'] + df['Interest GMV']
# df['Dex Volume DeBank'] = df['Trading Volume Debank'] + df['Borrowing Volume Debank']

In [14]:
df['max_test'] = df[['MAX Supply CMC', 'Total Supply CMC']].max(axis=1)
df['fully_diluted_mkt_cap'] = df.max_test * df.Price

del df['max_test']

In [15]:
df["total_revenue"] = df["Transaction Revenue"] + df["Interest Revenue"]
df["total_supply_revenue"] = df["Transaction Revenue Supply Side"] + df["Interest Revenue Supply Side"]
df["total_protocol_revenue"] = df["Interest Revenue Protocol"] + df["Transaction Revenue Protocol"]


In [16]:
df[["total_revenue", "total_supply_revenue", "total_protocol_revenue"]]

,total_revenue,total_supply_revenue,total_protocol_revenue
0,51058.960937,50173.411248,885.549689
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
109940,NaN,NaN,NaN
109941,NaN,NaN,NaN
109942,NaN,NaN,NaN
109943,NaN,NaN,NaN


In [17]:
df = df.sort_values('date')

In [18]:
cmc_symbol_name_df = pd.read_sql_query("select distinct Symbol, Name from g_master_data_joins order by Symbol, Name", __connect__())


In [19]:
cmc_symbol_name_df

,Symbol,Name
0,$ANRX,AnRKey X
1,0xBTC,0xBitcoin
2,1INCH,1inch
3,AAVE,Aave
4,ABBC,ABBC Coin
...,...,...
1236,ZNN,Zenon
1237,ZORA,Zoracles
1238,ZRX,0x
1239,ZT,ZBG Token


In [20]:
def calculate_nDay_revenue(df, days):
    column_name = f"total_revenue_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['total_revenue'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

In [21]:
# function for defi volume sums
# substitute for volume cmc sum
# change 'Defi Volume DeBank' to 'Defi Volume_max' 
def calculate_nDay_defi_volume(df, days):
    column_name = f"total_defi_volume_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['Defi Volume_max'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]

In [1]:
def calculate_nDay_change(df, days, column, column_name):
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df[column].pct_change(days)
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    
def calculate_nDay_std(df, days, column, column_name):
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df[column].rolling(days).std() / temp_df[column]
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

In [23]:
calculate_nDay_revenue(df, 7)
calculate_nDay_revenue(df, 30)
#calculate_nDay_trading_volume(df, 30)
calculate_nDay_defi_volume(df, 30)

# add 1, 3, 7, 30 day defi volume pct change here
# added column calculations
# 
calculate_nDay_change(df, 1, 'Defi Volume_max', 'Defi_volume_pct_chg_1d')
calculate_nDay_change(df, 3, 'Defi Volume_max', 'Defi_volume_pct_chg_3d')
calculate_nDay_change(df, 7, 'Defi Volume_max', 'Defi_volume_pct_chg_7d')
calculate_nDay_change(df, 30, 'Defi Volume_max', 'Defi_volume_pct_chg_30d')
#

In [24]:
df[df['Symbol'] == 'AAVE']['total_revenue_7d']

0         5.105896e+04
998       1.039415e+05
1833      1.514182e+05
2017      2.232560e+05
3048      2.762748e+05
              ...     
104325    2.118522e+06
105449    2.116004e+06
107071    1.824975e+06
108496    1.795100e+06
108821    1.470444e+06
Name: total_revenue_7d, Length: 108, dtype: float64

In [25]:
df[df['Symbol'] == 'AAVE']['total_defi_volume_30d']

0        NaN
998      NaN
1833     NaN
2017     NaN
3048     NaN
          ..
104325   NaN
105449   NaN
107071   NaN
108496   NaN
108821   NaN
Name: total_defi_volume_30d, Length: 108, dtype: float64

In [26]:
df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,total_revenue,total_supply_revenue,total_protocol_revenue,total_revenue_7d,total_revenue_30d,total_defi_volume_30d,Defi_volume_pct_chg_1d,Defi_volume_pct_chg_3d,Defi_volume_pct_chg_7d,Defi_volume_pct_chg_30d
0,2020-12-21,Aave,AAVE,81.311937,9.757233e+08,2.154891e+08,1.199975e+07,1.600000e+07,1.600000e+07,1.723929e+09,...,51058.960937,50173.411248,885.549689,51058.960937,51058.960937,NaN,NaN,NaN,NaN,NaN
438,2020-12-21,Filecoin,FIL,25.221774,1.124493e+09,3.038169e+08,4.458420e+07,2.000000e+09,4.458420e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
439,2020-12-21,FOAM,FOAM,0.014965,4.601263e+06,1.635864e+04,3.074771e+08,NaN,1.000000e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,2020-12-21,Fusion,FSN,0.183806,8.853502e+06,1.145879e+06,4.816764e+07,NaN,6.573740e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,2020-12-21,1irstcoin,FST,0.834601,2.918441e+07,5.696083e+04,3.496810e+07,NaN,1.000000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109192,2021-04-07,Chimpion,BNANA,3.837349,1.217491e+08,5.975214e+05,3.172739e+07,NaN,1.000000e+11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109191,2021-04-07,BitMart Token,BMX,0.096084,1.668377e+07,1.559358e+06,1.736370e+08,NaN,6.505790e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109190,2021-04-07,Bluzelle,BLZ,0.501143,1.431863e+08,4.115452e+07,2.857198e+08,NaN,5.000000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109196,2021-04-07,BOLT,BOLT,0.016083,1.594329e+07,6.403727e+05,9.912831e+08,NaN,9.950000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df.columns

Index(['date', 'Name', 'Symbol', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Debank', 'TVL Defipulse', 'Transaction Revenue',
       'Interest Revenue', 'Transaction Revenue Supply Side',
       'Interest Revenue Supply Side', 'Transaction Revenue Protocol',
       'Interest Revenue Protocol', 'Transaction GMV', 'Interest GMV',
       'Price/Sale', 'Users 24hr Debank', 'Transactions Count Debank',
       'Trading Volume Debank', 'Borrowing Volume Debank',
       'Users life-to-date duneanlytics', 'CoinUrl', 'Tags', 'TVL_max', 'GMV',
       'Defi Volume DeBank', 'fully_diluted_mkt_cap', 'total_revenue',
       'total_supply_revenue', 'total_protocol_revenue', 'total_revenue_7d',
       'total_revenue_30d', 'total_defi_volume_30d', 'Defi_volume_pct_chg_1d',
       'Defi_volume_pct_chg_3d', 'Defi_volume_pct_chg_7d',
       'Defi_volume_pct_chg_30d'],
      dtype='object')

In [28]:
# df['Marketcap CMC'] = df['Marketcap CMC'] + 0.01
# df['fully_diluted_mkt_cap'] = df['fully_diluted_mkt_cap'] + 0.01
# df['TVL Debank'] = df['TVL Debank'] + 0.01
# df['Users 24hr Debank'] = df['Users 24hr Debank'] + 0.01
# df['Transactions Count Debank'] = df['Transactions Count Debank'] + 0.01
# df['total_volume_30d'] = df['total_volume_30d'] + 0.01
# df['Trading Volume Debank'] = df['Trading Volume Debank'] + 0.01
# df['total_revenue'] = df['total_revenue'] + 0.01

In [29]:
# flip such that mc in numerator 
#df['VOL : MC'] = df['Volume CMC'] / df['Marketcap CMC']
df['MC : VOL'] = df['Marketcap CMC'] / df['Volume CMC'] 
# added column calculations
calculate_nDay_change(df, 1, 'MC : VOL', 'MC_VOL_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : VOL', 'MC_VOL_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : VOL', 'MC_VOL_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : VOL', 'MC_VOL_pct_chg_30d')
#

# flip such that mc in numerator
#df['VOL : DMC'] = df['Volume CMC'] / df['fully_diluted_mkt_cap']
df['DMC : VOL'] = df['fully_diluted_mkt_cap'] / df['Volume CMC']
# added column calculations
calculate_nDay_change(df, 1, 'DMC : VOL', 'DMC_VOL_pct_chg_1d')
calculate_nDay_change(df, 3, 'DMC : VOL', 'DMC_VOL_pct_chg_3d')
calculate_nDay_change(df, 7, 'DMC : VOL', 'DMC_VOL_pct_chg_7d')
calculate_nDay_change(df, 30, 'DMC : VOL', 'DMC_VOL_pct_chg_30d')
#



df['MC : TVL'] = df['Marketcap CMC'] / df['TVL_max']
# added column calculations
calculate_nDay_change(df, 1, 'MC : TVL', 'MC_TVL_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : TVL', 'MC_TVL_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : TVL', 'MC_TVL_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : TVL', 'MC_TVL_pct_chg_30d')
#


df['DMC : TVL'] = df['fully_diluted_mkt_cap'] / df['TVL_max']
# added column calculations
calculate_nDay_change(df, 1, 'DMC : TVL', 'DMC_TVL_pct_chg_1d')
calculate_nDay_change(df, 3, 'DMC : TVL', 'DMC_TVL_pct_chg_3d')
calculate_nDay_change(df, 7, 'DMC : TVL', 'DMC_TVL_pct_chg_7d')
calculate_nDay_change(df, 30, 'DMC : TVL', 'DMC_TVL_pct_chg_30d')
#


df['MC : users (1d)'] = df['Marketcap CMC'] / df['Users 24hr Debank']
# added column calculations
calculate_nDay_change(df, 1, 'MC : users (1d)', 'MC_users_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : users (1d)', 'MC_users_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : users (1d)', 'MC_users_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : users (1d)', 'MC_users_pct_chg_30d')
#




df['DMC : users (1d)'] = df['fully_diluted_mkt_cap'] / df['Users 24hr Debank']
# added column calculations
calculate_nDay_change(df, 1, 'DMC : users (1d)', 'DMC_users_pct_chg_1d')
calculate_nDay_change(df, 3, 'DMC : users (1d)', 'DMC_users_pct_chg_3d')
calculate_nDay_change(df, 7, 'DMC : users (1d)', 'DMC_users_pct_chg_7d')
calculate_nDay_change(df, 30, 'DMC : users (1d)', 'DMC_users_pct_chg_30d')
#



df['MC : transactions (1d)'] = df['Marketcap CMC'] / df['Transactions Count Debank']
# added column calculations
calculate_nDay_change(df, 1, 'MC : transactions (1d)', 'MC_transactions_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : transactions (1d)', 'MC_transactions_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : transactions (1d)', 'MC_transactions_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : transactions (1d)', 'MC_transactions_pct_chg_30d')
#

df['DMC : transactions (1d)'] = df['fully_diluted_mkt_cap'] / df['Transactions Count Debank']
# added column calculations
calculate_nDay_change(df, 1, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_1d')
calculate_nDay_change(df, 3, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_3d')
calculate_nDay_change(df, 7, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_7d')
calculate_nDay_change(df, 30, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_30d')
#

#df['MC : DEX Vol (1d)'] = df['Marketcap CMC'] / df['Trading Volume Debank']
# add
df['MC : Defi Vol (1d)'] = df['Marketcap CMC'] / df['Defi Volume_max']
# added column calculations
calculate_nDay_change(df, 1, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_30d')
#


df['MC : Defi Vol (30d)'] = df['Marketcap CMC'] / df['total_defi_volume_30d']
#df['DMC : DEX Vol (1d)'] = df['fully_diluted_mkt_cap'] / df['Trading Volume Debank']
#df['DMC : DEX Vol (30d)'] = df['fully_diluted_mkt_cap'] / df['total_volume_30d']



df['MC : revenue'] = df['Marketcap CMC'] / df['total_revenue']
# added column calculations
calculate_nDay_change(df, 1, 'MC : revenue', 'MC_revenue_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : revenue', 'MC_revenue_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : revenue', 'MC_revenue_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : revenue', 'MC_revenue_pct_chg_30d')
#

df['DMC : revenue'] = df['fully_diluted_mkt_cap'] / df['total_revenue']
# added column calculations
calculate_nDay_change(df, 1, 'DMC : revenue', 'DMC_revenue_pct_chg_1d')
calculate_nDay_change(df, 3, 'DMC : revenue', 'DMC_revenue_pct_chg_3d')
calculate_nDay_change(df, 7, 'DMC : revenue', 'DMC_revenue_pct_chg_7d')
calculate_nDay_change(df, 30, 'DMC : revenue', 'DMC_revenue_pct_chg_30d')
#


# df['MC : policies sold'] = df['Marketcap CMC'] / df['abc']
# df['DMC : policies sold'] = df['fully_diluted_mkt_cap'] / df['abc']
# df['MC : premium'] = df['Marketcap CMC'] / df['abc']
# df['DMC : premium'] = df['fully_diluted_mkt_cap'] / df['abc']

In [30]:
df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,DMC_transactions_pct_chg_7d,DMC_transactions_pct_chg_30d,MC : Defi Vol (1d),MC_defi_volume_pct_chg_1d,MC_defi_volume_pct_chg_3d,MC_defi_volume_pct_chg_7d,MC_defi_volume_pct_chg_30d,MC : Defi Vol (30d),MC : revenue,DMC : revenue
0,2020-12-21,Aave,AAVE,81.311937,9.757233e+08,2.154891e+08,1.199975e+07,1.600000e+07,1.600000e+07,1.723929e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19109.736101,25480.169805
438,2020-12-21,Filecoin,FIL,25.221774,1.124493e+09,3.038169e+08,4.458420e+07,2.000000e+09,4.458420e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
439,2020-12-21,FOAM,FOAM,0.014965,4.601263e+06,1.635864e+04,3.074771e+08,NaN,1.000000e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,2020-12-21,Fusion,FSN,0.183806,8.853502e+06,1.145879e+06,4.816764e+07,NaN,6.573740e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,2020-12-21,1irstcoin,FST,0.834601,2.918441e+07,5.696083e+04,3.496810e+07,NaN,1.000000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109192,2021-04-07,Chimpion,BNANA,3.837349,1.217491e+08,5.975214e+05,3.172739e+07,NaN,1.000000e+11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109191,2021-04-07,BitMart Token,BMX,0.096084,1.668377e+07,1.559358e+06,1.736370e+08,NaN,6.505790e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109190,2021-04-07,Bluzelle,BLZ,0.501143,1.431863e+08,4.115452e+07,2.857198e+08,NaN,5.000000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109196,2021-04-07,BOLT,BOLT,0.016083,1.594329e+07,6.403727e+05,9.912831e+08,NaN,9.950000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
calculate_nDay_change(df, 1, 'Price', 'Price_pct_chg_1d')
calculate_nDay_change(df, 3, 'Price', 'Price_pct_chg_3d')
calculate_nDay_change(df, 7, 'Price', 'Price_pct_chg_7d')
calculate_nDay_change(df, 30, 'Price', 'Price_pct_chg_30d')
calculate_nDay_std(df, 7, 'Price', 'Price_7day_volatility_pct')
calculate_nDay_std(df, 30, 'Price', 'Price_30day_volatility_pct')
calculate_nDay_std(df, 90, 'Price', 'Price_90day_volatility_pct')

In [32]:
calculate_nDay_change(df, 1, 'Marketcap CMC', 'MC_pct_chg_1d')
calculate_nDay_change(df, 3, 'Marketcap CMC', 'MC_pct_chg_3d')
calculate_nDay_change(df, 7, 'Marketcap CMC', 'MC_pct_chg_7d')
calculate_nDay_change(df, 30, 'Marketcap CMC', 'MC_pct_chg_30d')
calculate_nDay_std(df, 30, 'Marketcap CMC', 'MC_30day_volatility_pct')
calculate_nDay_std(df, 90, 'Marketcap CMC', 'MC_90day_volatility_pct')

In [33]:
calculate_nDay_change(df, 1, 'fully_diluted_mkt_cap', 'DMC_pct_chg_1d')
calculate_nDay_change(df, 3, 'fully_diluted_mkt_cap', 'DMC_pct_chg_3d')
calculate_nDay_change(df, 7, 'fully_diluted_mkt_cap', 'DMC_pct_chg_7d')
calculate_nDay_change(df, 30, 'fully_diluted_mkt_cap', 'DMC_pct_chg_30d')
calculate_nDay_std(df, 30, 'fully_diluted_mkt_cap', 'DMC_30day_volatility_pct')
calculate_nDay_std(df, 90, 'fully_diluted_mkt_cap', 'DMC_90day_volatility_pct')

In [34]:
calculate_nDay_change(df, 1, 'Circulating supply CMC', 'CS_pct_chg_1d')
calculate_nDay_change(df, 3, 'Circulating supply CMC', 'CS_pct_chg_d')
calculate_nDay_change(df, 7, 'Circulating supply CMC', 'CS_pct_chg_7d')
calculate_nDay_change(df, 30, 'Circulating supply CMC', 'CS_pct_chg_30d')
calculate_nDay_std(df, 30, 'Circulating supply CMC', 'CS_30day_volatility_pct')
calculate_nDay_std(df, 90, 'Circulating supply CMC', 'CS_90day_volatility_pct')

In [35]:
calculate_nDay_change(df, 1, 'Volume CMC', 'Volume_pct_chg_1d')
calculate_nDay_change(df, 3, 'Volume CMC', 'Volume_pct_chg_3d')
calculate_nDay_change(df, 7, 'Volume CMC', 'Volume_pct_chg_7d')
calculate_nDay_change(df, 30, 'Volume CMC', 'Volume_pct_chg_30d')
calculate_nDay_std(df, 30, 'Volume CMC', 'Volume_30day_volatility_pct')
calculate_nDay_std(df, 90, 'Volume CMC', 'Volume_90day_volatility_pct')

In [36]:
calculate_nDay_change(df, 1, 'TVL_max', 'TVL_pct_chg_1d')
calculate_nDay_change(df, 3, 'TVL_max', 'TVL_pct_chg_3d')
calculate_nDay_change(df, 7, 'TVL_max', 'TVL_pct_chg_7d')
calculate_nDay_change(df, 30, 'TVL_max', 'TVL_pct_chg_30d')
calculate_nDay_std(df, 30, 'TVL_max', 'TVL_30day_volatility_pct')
calculate_nDay_std(df, 90, 'TVL_max', 'TVL_90day_volatility_pct')

In [37]:
calculate_nDay_change(df, 1, 'total_revenue', 'Rev_pct_chg_1d')
calculate_nDay_change(df, 3, 'total_revenue', 'Rev_pct_chg_3d')
calculate_nDay_change(df, 7, 'total_revenue', 'Rev_pct_chg_7d')
calculate_nDay_change(df, 30, 'total_revenue', 'Rev_pct_chg_30d')
calculate_nDay_std(df, 30, 'total_revenue', 'Rev_30day_volatility_pct')
calculate_nDay_std(df, 90, 'total_revenue', 'Rev_90day_volatility_pct')

In [38]:
# moved pct change to next to ratio so commenting these out

#calculate_nDay_change(df, 1, 'MC : TVL', 'MC : TVL_pct_chg_1d')
#calculate_nDay_change(df, 3, 'MC : TVL', 'MC : TVL_pct_chg_3d')
#calculate_nDay_change(df, 7, 'MC : TVL', 'MC : TVL_pct_chg_7d')
#calculate_nDay_change(df, 30, 'MC : TVL', 'MC : TVL_pct_chg_30d')
# commented out below 
# calculate_nDay_std(df, 30, 'Circulating supply CMC', 'Rev σ 30d')
# calculate_nDay_std(df, 90, 'Circulating supply CMC', 'Rev σ 90d')

In [39]:
# moved pct change to next to ratio so commenting these out

#calculate_nDay_change(df, 1, 'DMC : TVL', 'DMC : TVL_pct_chg_1d')
#calculate_nDay_change(df, 3, 'DMC : TVL', 'DMC : TVL_pct_chg_3d')
#calculate_nDay_change(df, 7, 'DMC : TVL', 'DMC : TVL_pct_chg_7d')
#calculate_nDay_change(df, 30, 'DMC : TVL', 'DMC : TVL_pct_chg_30d')
# calculate_nDay_std(df, 30, 'Circulating supply CMC', 'CS σ 30d')
# calculate_nDay_std(df, 90, 'Circulating supply CMC', 'CS σ 90d')

In [40]:
df['Annual revenue (estimate)'] = df['total_revenue_30d'] * (1 + df['Rev_pct_chg_30d'])*12

In [41]:
df['Annual revenue (estimate)'].unique()

array([           nan, 0.00000000e+00, 2.02762697e+08, 1.79198565e+08,
       1.55136950e+08, 9.82067771e+07, 1.40273154e+08, 1.73302609e+08,
       1.18109496e+08, 1.30405992e+08, 7.82277420e+07, 1.06151542e+08,
       4.83362838e+07, 3.54432943e+07, 3.57654763e+07, 2.42611508e+07,
       3.62822740e+07, 9.78291443e+08, 1.58607172e+08, 1.24916389e+08,
       1.08150777e+08, 8.31272407e+07, 1.67808110e+08, 1.82386531e+08,
       1.12688990e+08, 1.85816255e+08, 2.53641482e+08, 1.99097480e+08,
       1.94388793e+08, 2.48472942e+08, 2.68241292e+08, 2.29419238e+08,
       2.08560426e+08, 1.82448061e+08, 2.69880318e+08, 3.28002294e+08,
       2.74389703e+08, 1.67887901e+08, 1.79530880e+08, 2.21814828e+08,
       1.98902733e+08, 2.11631211e+08, 1.79019530e+08, 2.21742618e+08,
       1.75270421e+08, 2.43850185e+08, 1.51511963e+08, 1.32118698e+07,
       1.09762841e+08, 2.81240136e+08, 3.09942448e+08, 3.66379107e+08,
       2.37816658e+08, 1.61882767e+08, 1.22663961e+08, 1.10058266e+08,
      

In [42]:
df.columns

Index(['date', 'Name', 'Symbol', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Debank',
       ...
       'TVL_pct_chg_30d', 'TVL_30day_volatility_pct',
       'TVL_90day_volatility_pct', 'Rev_pct_chg_1d', 'Rev_pct_chg_3d',
       'Rev_pct_chg_7d', 'Rev_pct_chg_30d', 'Rev_30day_volatility_pct',
       'Rev_90day_volatility_pct', 'Annual revenue (estimate)'],
      dtype='object', length=125)

In [43]:
df[(df['Symbol'] == 'ZRX')]['GMV']

662       6.533663e+07
997       5.306118e+07
1832      6.193404e+07
2701      6.455556e+07
3046      5.017659e+07
              ...     
105337    2.042772e+08
106570    1.164772e+08
107068             NaN
108493    1.522847e+08
109640             NaN
Name: GMV, Length: 108, dtype: float64

In [44]:
df.to_csv('master_data_ratios_and_percentages.csv')

In [45]:
df.shape

(109945, 125)

In [46]:
required_tags = ['binance-smart-chain','polkadot-ecosystem','ethereum','cosmos-ecosystem','polkadot','solana-ecosystem','binance-chain','avalanche-ecosystem','heco','layer-2','eos','sidechain','cosmos','tron','defi','yield-farming','decentralized-exchange','governance','dao','gaming','stablecoin','asset-management','centralized-exchange','staking','yield-aggregator','oracles','lending-borowing','dex','options','launchpad','stablecoin-algorithmically-stabilized']
required_tags.sort()
required_tags

['asset-management',
 'avalanche-ecosystem',
 'binance-chain',
 'binance-smart-chain',
 'centralized-exchange',
 'cosmos',
 'cosmos-ecosystem',
 'dao',
 'decentralized-exchange',
 'defi',
 'dex',
 'eos',
 'ethereum',
 'gaming',
 'governance',
 'heco',
 'launchpad',
 'layer-2',
 'lending-borowing',
 'options',
 'oracles',
 'polkadot',
 'polkadot-ecosystem',
 'sidechain',
 'solana-ecosystem',
 'stablecoin',
 'stablecoin-algorithmically-stabilized',
 'staking',
 'tron',
 'yield-aggregator',
 'yield-farming']

In [47]:
data_df = df
data_df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,TVL_pct_chg_30d,TVL_30day_volatility_pct,TVL_90day_volatility_pct,Rev_pct_chg_1d,Rev_pct_chg_3d,Rev_pct_chg_7d,Rev_pct_chg_30d,Rev_30day_volatility_pct,Rev_90day_volatility_pct,Annual revenue (estimate)
0,2020-12-21,Aave,AAVE,81.311937,9.757233e+08,2.154891e+08,1.199975e+07,1.600000e+07,1.600000e+07,1.723929e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,2020-12-21,Filecoin,FIL,25.221774,1.124493e+09,3.038169e+08,4.458420e+07,2.000000e+09,4.458420e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
439,2020-12-21,FOAM,FOAM,0.014965,4.601263e+06,1.635864e+04,3.074771e+08,NaN,1.000000e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,2020-12-21,Fusion,FSN,0.183806,8.853502e+06,1.145879e+06,4.816764e+07,NaN,6.573740e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,2020-12-21,1irstcoin,FST,0.834601,2.918441e+07,5.696083e+04,3.496810e+07,NaN,1.000000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109192,2021-04-07,Chimpion,BNANA,3.837349,1.217491e+08,5.975214e+05,3.172739e+07,NaN,1.000000e+11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109191,2021-04-07,BitMart Token,BMX,0.096084,1.668377e+07,1.559358e+06,1.736370e+08,NaN,6.505790e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109190,2021-04-07,Bluzelle,BLZ,0.501143,1.431863e+08,4.115452e+07,2.857198e+08,NaN,5.000000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109196,2021-04-07,BOLT,BOLT,0.016083,1.594329e+07,6.403727e+05,9.912831e+08,NaN,9.950000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df = pd.DataFrame(data_df['Tags'].copy())
df

,Tags
0,"defi,dao,yield-farming,three-arrows-capital-po..."
438,"mineable,distributed-computing,filesharing,sto..."
439,"platform,cosmos-ecosystem,ai-big-data,placehol..."
440,"platform,defi,payments"
441,services
...,...
109192,
109191,marketplace
109190,"platform,cosmos-ecosystem,distributed-computin..."
109196,


In [49]:
required_tags_map = {}
i = 0
for tag in required_tags:
    required_tags_map[tag] = i
    i = i + 1

In [50]:
required_tags_map

{'asset-management': 0,
 'avalanche-ecosystem': 1,
 'binance-chain': 2,
 'binance-smart-chain': 3,
 'centralized-exchange': 4,
 'cosmos': 5,
 'cosmos-ecosystem': 6,
 'dao': 7,
 'decentralized-exchange': 8,
 'defi': 9,
 'dex': 10,
 'eos': 11,
 'ethereum': 12,
 'gaming': 13,
 'governance': 14,
 'heco': 15,
 'launchpad': 16,
 'layer-2': 17,
 'lending-borowing': 18,
 'options': 19,
 'oracles': 20,
 'polkadot': 21,
 'polkadot-ecosystem': 22,
 'sidechain': 23,
 'solana-ecosystem': 24,
 'stablecoin': 25,
 'stablecoin-algorithmically-stabilized': 26,
 'staking': 27,
 'tron': 28,
 'yield-aggregator': 29,
 'yield-farming': 30}

In [51]:
all_symbol_tags = []
for index, row in df.iterrows():
    tag = [''] * (len(required_tags))
    tags = row['Tags']
    tags = tags.split(',')
    for t in tags:
        index = required_tags_map[t] if t in required_tags_map else -1
        if index > -1:
            tag[index] = 1
    all_symbol_tags.append(tag)
        

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
all_symbol_tags

In [ ]:
columns = []
columns.extend(required_tags)
columns

In [ ]:
df2 = pd.DataFrame(all_symbol_tags, columns = columns)
df2

In [ ]:
data_df = data_df.join(df2)
del data_df['Tags']
del data_df['CoinUrl']

In [ ]:
data_df

In [ ]:
data_df.to_csv('master_data_ratios_and_percentages.csv')

In [ ]:
data_df.shape

In [ ]:
data_df.columns

In [ ]:
s = 'qw'
symbol = 'asd'

In [ ]:
f"{s},'{str(symbol)}'"